CODE FOR TRAINING THE MODEL

LIBRARY IMPORTS

In [20]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

LOADING DATA

In [5]:
data_dirs = {
    "URTI": r"C:\Users\Jiary\Documents\GitHub\ML\Spectograms\URTI",
    "Healthy": r"C:\Users\Jiary\Documents\GitHub\ML\Spectograms\Healthy",
    "COPD": r"C:\Users\Jiary\Documents\GitHub\ML\Spectograms\COPD",
    "Bronchiectasis": r"C:\Users\Jiary\Documents\GitHub\ML\Spectograms\Bronchiectasis",
    "Pneumonia": r"C:\Users\Jiary\Documents\GitHub\ML\Spectograms\Pneumonia",
    "Bronchiolitis": r"C:\Users\Jiary\Documents\GitHub\ML\Spectograms\Bronchiolitis"
}

for disease, path in data_dirs.items():
    print(f"Loaded {disease} data directory: {path}")

# Load data and labels
X = []  # Spectrograms
y = []  # Labels

for label, folder_path in data_dirs.items():
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".npy"):
            file_path = os.path.join(folder_path, file_name)
            data = np.load(file_path)  # Shape: (128, 128, 1)
            X.append(data)
            y.append(label)

# Convert to numpy arrays
X = np.array(X)  # Shape: (n_samples, 128, 128, 1)
y = np.array(y)  # Shape: (n_samples,)

Loaded URTI data directory: C:\Users\Jiary\Documents\GitHub\ML\Spectograms\URTI
Loaded Healthy data directory: C:\Users\Jiary\Documents\GitHub\ML\Spectograms\Healthy
Loaded COPD data directory: C:\Users\Jiary\Documents\GitHub\ML\Spectograms\COPD
Loaded Bronchiectasis data directory: C:\Users\Jiary\Documents\GitHub\ML\Spectograms\Bronchiectasis
Loaded Pneumonia data directory: C:\Users\Jiary\Documents\GitHub\ML\Spectograms\Pneumonia
Loaded Bronchiolitis data directory: C:\Users\Jiary\Documents\GitHub\ML\Spectograms\Bronchiolitis


ENCODING LABELS

In [11]:
 # Convert labels to integers
label_encoder = LabelEncoder()
label_encoder.fit(y)  # Fit on all labels in the dataset
# Step 2: Encode the labels
y_encoded = label_encoder.transform(y)
# Example mapping
print(f'Total spectrograms: {len(X)}')
print("Label mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

Total spectrograms: 8825
Label mapping: {'Bronchiectasis': 0, 'Bronchiolitis': 1, 'COPD': 2, 'Healthy': 3, 'Pneumonia': 4, 'URTI': 5}


SPLIT TRAIN-EVALUATE-TEST DATA

In [16]:
# Stratified split into training (70%) and temporary set (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
# Print sizes
print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples\n\n")
# Function to count samples per class
def count_samples(y_encoded, label_encoder):
    unique, counts = np.unique(y_encoded, return_counts=True)
    for cls, count in zip(unique, counts):
        print(f"Class: {label_encoder.inverse_transform([cls])[0]}, Samples: {count}")

# Print distribution in training set
print("Training Set Distribution:")
count_samples(y_train, label_encoder)

# Print distribution in validation set
print("\nValidation Set Distribution:")
count_samples(y_val, label_encoder)

# Print distribution in testing set
print("\nTesting Set Distribution:")
count_samples(y_test, label_encoder)


Training set: 6177 samples
Validation set: 1324 samples
Testing set: 1324 samples


Training Set Distribution:
Class: Bronchiectasis, Samples: 349
Class: Bronchiolitis, Samples: 364
Class: COPD, Samples: 4022
Class: Healthy, Samples: 498
Class: Pneumonia, Samples: 490
Class: URTI, Samples: 454

Validation Set Distribution:
Class: Bronchiectasis, Samples: 75
Class: Bronchiolitis, Samples: 78
Class: COPD, Samples: 862
Class: Healthy, Samples: 107
Class: Pneumonia, Samples: 105
Class: URTI, Samples: 97

Testing Set Distribution:
Class: Bronchiectasis, Samples: 75
Class: Bronchiolitis, Samples: 78
Class: COPD, Samples: 862
Class: Healthy, Samples: 107
Class: Pneumonia, Samples: 105
Class: URTI, Samples: 97


CNN-MODEL

In [21]:
# Define the CNN model
model = Sequential([
    # Convolutional layer
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    
    # Second convolutional layer
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Third convolutional layer
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Flatten the output
    Flatten(),
    
    # Fully connected layers
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout for regularization
    Dense(64, activation='relu'),
    
    # Output layer (6 classes for your dataset)
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

C:\Users\Jiary\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,312,710 (12.64 MB)

 Trainable params: 3,312,710 (12.64 MB)

 Non-trainable params: 0 (0.00 B)